<io>
사업보고서 전체에 대해서 보는 것이 필요하나, 이미 이전의 단어 추출 기반으로 산업을 분류한 것과 유사하게, <br>
business description에서 나온 문장 단위에서 token화를 해 준 다음 unigrams와 bigrams를 기반으로 filling date 이후 CAR 을 예측해보고자 한다. <br>
대조군으로 Loughran and McDonald 사전 기반으로 긍정, 부정, 긍정부정 비율 등과 비교하여 결과의 정확도를 보고자 한다. <br>


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk import word_tokenize, pos_tag
from collections import Counter
import logging
import sys, traceback
import multiprocessing
import time

logging.basicConfig(level=logging.ERROR)

def textmaking(start_num, end_num): #Business description 부분에서 단어 추출하기 위한 함수
    try:
        df = pd.read_csv('ALL_df.csv', index_col=0)
        print(start_num, '~', end_num) #멀티프로세스가 잘 되고 있는지 확인하기 위한 용도

        #한번만 설치
        #nltk.download('punkt')
        #nltk.download('averaged_perceptron_tagger')

        #word_df = pd.DataFrame()
        word_df = pd.DataFrame(columns=['FY', 'CIK', 'NAME', 'word', 'count'])
        #ii=0
        #if len(word_df) != 0:
        #    ii=word_df.loc[len(word_df)-1, 'i']+1

        for i in range(start_num, end_num):
            err = 1 #단어 추출과는 다르게 문장별로 할껀데, 이전 예외처리들이 하나하나 다시 점검해야되서 귀찮아서 걍 예외처리한것들은 싹다 빼고 할꺼임
            intel = 0
            morgan = 0
            key = 0
            ge=0
            gen=0
            att=0
            city=0
            edison = 0
            cardinal=0
            entergy=0
            forrester =0
            wy=0
            MRTX = 0
            clean = 0
            freddie = 0
            fy = df['FY'][i]
            cik= df['CIK'][i]
            name= df['NAME'][i].replace('/', '-')
            name= name.replace('\\', '-')
            ACCN = df['ACCN'][i]
            ACCN1 = df['ACCN'][i].replace('-', '')
            url = 'https://www.sec.gov/Archives/edgar/data/'+str(cik)+'/'+str(ACCN1)+'/'+str(ACCN)+'.txt'

            headers = {'User-Agent': 'Mozilla'}
            res = requests.get(url, headers=headers)
            text=[]
            if res.status_code == 200:
                # BeautifulSoup을 사용하여 HTML 파싱
                if i>=256 and i<= 258:
                    soup = BeautifulSoup(res.text, 'html.parser')
                    intel = 1
                    err = 0
                elif (i>=319 and i<=321) or (i>=2010 and i<=2016): #overview 이후 financial 까지 해야되는 경우
                    soup = BeautifulSoup(res.text, 'lxml')
                    morgan = 1
                    err = 0
                elif (i>=344 and i<=349) or (i>=1922 and i<=1925): #about 뭐시기로 끊고 바로 legal 까지 대문자 형태로 구분이 되는 경우
                    if i<349:
                        first = res.text.find("ABOUT GENERAL ELECTRIC")
                    elif i<1925:
                        first = res.text.find("ABOUT MOLINA HEALTHCARE")
                    end = res.text.find("LEGAL PROCEEDINGS")
                    tt = res.text[first:end]
                    soup = BeautifulSoup(tt, 'lxml')
                    ge=1
                    err = 0
                elif (i>=595 and i<= 601):
                    soup = BeautifulSoup(res.text, 'lxml')
                    city = 1
                    err = 0
                    if i == 595:
                        city = 2
                        err = 0
                elif (i>=1710 and i<=1715): #EDISON 형식, CORPORATE STRUCTURE, INDUSTRY AND OTHER INFORMATION 부터 staff 까지
                    soup = BeautifulSoup(res.text, 'lxml')
                    edison = 1
                    err = 0
                elif (i>=1753 and i<=1757) or (i>=2031 and i<=2033):#business general 꼴
                    soup = BeautifulSoup(res.text, 'lxml')
                    cardinal=1
                    err = 0
                elif (i>=1877 and i<=1883): #OUrbusiness 형식으로 되어있어서 구분이 힘든 애들
                    soup = BeautifulSoup(res.text, 'lxml')
                    wy = 1
                    err = 0
                elif (i>=2003 and i<=2009): #entergy 형식들, 아예 형식이 너무 다름
                    try: #lxml이 안먹는 애들
                        soup = BeautifulSoup(res.text, 'html.parser')
                    except:
                        soup = BeautifulSoup(res.text, 'lxml')

                    entergy = 1
                    err = 0
                elif (i>=5214 and i<=5220):
                    try: #lxml이 안먹는 애들
                        soup = BeautifulSoup(res.text, 'html.parser')
                    except:
                        soup = BeautifulSoup(res.text, 'lxml')
                    MRTX = 1
                    err = 0
                elif i == 181:
                    first = res.text.find("BUSINESS SUMMARY")
                    end = res.text.find("LEGAL PROCEEDINGS")
                    tt = res.text[first:end]
                    soup = BeautifulSoup(tt, 'lxml')
                    ge=1
                    err = 0
                elif i== 7309:
                    first = res.text.find("THE COMPANY")
                    end = res.text.find("LEGAL PROCEEDINGS")
                    tt = res.text[first:end]
                    soup = BeautifulSoup(tt, 'lxml')
                    ge=1
                    err = 0
                elif (i>=9933 and i<= 9937): #part i general 로 시작하는 형태
                    soup = BeautifulSoup(res.text, 'lxml')
                    forrester = 1
                    err = 0
                elif (i>=10600 and i <= 10604):
                    soup = BeautifulSoup(res.text, 'html.parser')
                    freddie = 1
                    err = 0
                elif (i>=14100 and i<=14101):
                    soup = BeautifulSoup(res.text, 'html.parser')
                    clean = 1
                    err = 0
                elif (i>=14450 and i<=14451):
                    soup = BeautifulSoup(res.text, 'html.parser')
                    gen = 1
                    err = 0
                else:
                    try: #lxml이 안먹는 애들
                        soup = BeautifulSoup(res.text, 'html.parser')
                    except:
                        soup = BeautifulSoup(res.text, 'lxml')
                
                if err == 1:
                    # 모든 스크립트와 스타일 태그 제거
                    for script in soup(["script", "style"]):
                        script.extract()

                    # 텍스트 추출
                    text = soup.get_text().strip()
                    # 텍스트에서 구두점 및 공백 제거 
                    text = text.replace('%', 'percent')
                    text = re.sub(r'[^\w\s\.]',' ', text)
                    text = re.sub(r'\xa0', ' ', text)    
                    text = re.sub(r'\n', ' ', text)
                    text = re.sub(r'\s+', ' ', text)
                    text = text.lower()

                    if intel == 1:
                        start_index = text.find("introduction to our business")
                        first_met = text.find("availability of company information") #처음으로 만나는 끝 직전
                        if ((start_index - first_met) < 0) and ((start_index - first_met) > -1000): #목차에서 서로 나타난다는 뜻임 unresolved 항목은 어떻게든 반드시 목차에선 저 문구대로 나올 예정
                            #슬라이스 후 item1 business 찾고 unresovlved 그 이후 찾기
                            text = text[start_index+len('item'):]
                            start_index = text.find("introduction to our business")
                            text = text[start_index+len('item'):]
                            end_index = text.find("availability of company information") #처음으로 만나는 끝 직전
                            text=text[:end_index]
                           # print('완료 : ', url, len(text))
                            key = 1
                        elif ((start_index - first_met) > 0) or ((start_index - first_met) < -1000): #목차에서 item1 business 형태가 아니라는 뜻임 아마 바로 항목으로 갈꺼임 / -1000은 목차가 없는 경우 방지 위해서 즉 바로 item1 나오는거
                            #슬라이스 후 바로 unresolved 찾기
                            text = text[start_index+len('item'):]
                            start_index = text.find("introduction to our business")
                            text = text[start_index+len('item'):]
                            end_index = text.find("availability of company information") #처음으로 만나는 끝 직전
                            text=text[:end_index]
                            #print('완료 : ', url, len(text))
                            key = 1
                    elif i == 2023:
                        start_index = text.find("business overview")
                        first_met = text.find("selected financial data") #처음으로 만나는 끝 직전
                        if ((start_index - first_met) < 0) and ((start_index - first_met) > -1000): #목차에서 서로 나타난다는 뜻임 unresolved 항목은 어떻게든 반드시 목차에선 저 문구대로 나올 예정
                            #슬라이스 후 item1 business 찾고 unresovlved 그 이후 찾기
                            text = text[start_index+len('item'):]
                            start_index = text.find("business overview")
                            text = text[start_index+len('item'):]
                            end_index = text.find("selected financial data") #처음으로 만나는 끝 직전
                            text=text[:end_index]
                            #print('완료 : ', url, len(text))
                            key = 1
                        elif ((start_index - first_met) > 0) or ((start_index - first_met) < -1000): #목차에서 item1 business 형태가 아니라는 뜻임 아마 바로 항목으로 갈꺼임 / -1000은 목차가 없는 경우 방지 위해서 즉 바로 item1 나오는거
                            #슬라이스 후 바로 unresolved 찾기
                            text = text[start_index+len('item'):]
                            start_index = text.find("overview")
                            text = text[start_index+len('item'):]
                            end_index = text.find("selected financial datas") #처음으로 만나는 끝 직전
                            text=text[:end_index]
                            #print('완료 : ', url, len(text))
                            key = 1
                    elif (i >= 15680 and i<= 15685): #갑자기 얘 혼자만 overview 부터 시작함
                        start_index = text.find("overview")
                        text = text[start_index+len('item'):]
                        end_index = text.find("unresolved staff comments")
                        text=text[:end_index]
                        #print('완료 : ', url, len(text))
                        key = 1
                    elif gen == 1:
                        start_index = text.find('company background')
                        text = text[start_index:]
                        end_index = text.find('unresolved staff comments')
                        text = text[:end_index]
                        key = 1
                    elif morgan == 1:
                        start_index = text.find("overview")
                        first_met = text.find("selected financial data") #처음으로 만나는 끝 직전
                        if ((start_index - first_met) < 0) and ((start_index - first_met) > -1000): #목차에서 서로 나타난다는 뜻임 unresolved 항목은 어떻게든 반드시 목차에선 저 문구대로 나올 예정
                            #슬라이스 후 item1 business 찾고 unresovlved 그 이후 찾기
                            text = text[start_index+len('item'):]
                            start_index = text.find("overview")
                            text = text[start_index+len('item'):]
                            end_index = text.find("selected financial data") #처음으로 만나는 끝 직전
                            text=text[:end_index]
                            #print('완료 : ', url, len(text))
                            key = 1
                        elif ((start_index - first_met) > 0) or ((start_index - first_met) < -1000): #목차에서 item1 business 형태가 아니라는 뜻임 아마 바로 항목으로 갈꺼임 / -1000은 목차가 없는 경우 방지 위해서 즉 바로 item1 나오는거
                            #슬라이스 후 바로 unresolved 찾기
                            text = text[start_index+len('item'):]
                            start_index = text.find("overview")
                            text = text[start_index+len('item'):]
                            end_index = text.find("selected financial datas") #처음으로 만나는 끝 직전
                            text=text[:end_index]
                            #print('완료 : ', url, len(text))
                            key = 1
                    elif MRTX == 1:
                        pattern = r"business\s?overview"
                        match = re.search(pattern, text)
                        if match:
                            start_index=match.end()
                            end_index = text[30000:].find("unresolved staff comments") #처음으로 만나는 끝 직전
                            if end_index == -1:
                                end_index = text[30000:].find("unresolved sec staff comments")
                                if end_index== -1:
                                    end_index = text[30000:].find("legal proceedings") 
                            text=text[:end_index+30000]
                            #print('완료 : ', url, len(text))
                            key = 1
                    elif edison == 1:
                        start_index = text.find('corporate structure')
                        text=text[start_index:]
                        end_index = text[30000:].find("unresolved staff comments") #처음으로 만나는 끝 직전
                        if end_index == -1:
                            end_index = text[30000:].find("unresolved sec staff comments")
                            if end_index== -1:
                                end_index = text[30000:].find("legal proceedings") 
                        text=text[:end_index+30000]
                        #print('완료 : ', url, len(text))
                        key = 1
                    elif cardinal == 1:
                        pattern = r"business\s?general"
                        match = re.search(pattern, text)
                        if match:
                            start_index = match.end()
                            first_met = text.find("legal proceedings") #처음으로 만나는 끝 직전
                            if first_met == -1:
                                first_met = text.find("legal proceedings")
                            if ((start_index - first_met) < 0) and ((start_index - first_met) > -1000): #목차에서 서로 나타난다는 뜻임 unresolved 항목은 어떻게든 반드시 목차에선 저 문구대로 나올 예정
                                #슬라이스 후 item1 business 찾고 unresovlved 그 이후 찾기
                                text = text[start_index+len('item'):]
                                match = re.search(pattern, text)
                                if match:
                                    start_index = match.end()
                                    text = text[start_index+len('item'):]
                                    end_index = text[30000:].find("legal proceedings") #처음으로 만나는 끝 직전
                                    text=text[:end_index+30000]
                                    #print('완료 : ', url, len(text))
                                    key = 1
                            elif ((start_index - first_met) > 0) or ((start_index - first_met) < -1000): #목차에서 item1 business 형태가 아니라는 뜻임 아마 바로 항목으로 갈꺼임 / -1000은 목차가 없는 경우 방지 위해서 즉 바로 item1 나오는거
                                #슬라이스 후 바로 unresolved 찾기
                                text = text[start_index+len('item'):]
                                end_index = text[30000:].find("legal proceedings") #처음으로 만나는 끝 직전
                                text=text[:end_index+30000]
                                #print('완료 : ', url, len(text))
                                key = 1
                    elif ge == 1:
                        key = 1
                        #print('완료 : ', url, len(text))
                    elif forrester == 1:
                        pattern = r"part\s?i\s?general"
                        match = re.search(pattern, text)
                        if match:
                            start_index=match.end()
                            text = text[start_index+len('item'):]
                            end_index = text[30000:].find("unresolved staff comments") #처음으로 만나는 끝 직전
                            if end_index == -1:
                                end_index = text[30000:].find("unresolved sec staff comments")
                                if end_index== -1:
                                    end_index = text[30000:].find("legal proceedings") 
                            text=text[:end_index+30000]
                            #print('완료 : ', url, len(text))
                            key = 1
                    elif wy == 1:
                        start_index = text.find('our businesswe are')
                        text=text[start_index:]
                        end_index = text[30000:].find("unresolved staff comments") #처음으로 만나는 끝 직전
                        if end_index == -1:
                            end_index = text[30000:].find("unresolved sec staff comments")
                            if end_index== -1:
                                end_index = text[30000:].find("legal proceedings") 
                        text=text[:end_index+30000]
                        #print('완료 : ', url, len(text))
                        key = 1
                    elif city > 0:
                        start_index = text.find('citigroups history')
                        text=text[start_index:]
                        if city == 2:
                            end_index = text.find('risk managementoverview')
                        elif city == 1:
                            end_index = text.find('managing global risk table of contents')
                        text=text[:end_index]
                        key = 1
                        #print('완료 : ', url, len(text)) 
                    elif freddie == 1:
                        start_index = text.find('freddie mac is a gse')
                        text=text[start_index:]
                        end_index = text.find('we are involved as a party')
                        text=text[:end_index]
                        key = 1
                        #print('완료 : ', url, len(text)) 
                    elif entergy == 1:
                        text = re.sub(r'font|style', ' ', text)
                        start_index = text.find('entergy is an integrated energy company')
                        text = text[start_index:]
                        end_index = text.find('entergy arkansas inc and subsidiaries')
                        text = text[:end_index]
                        #print('완료 : ', url, len(text))
                        key = 1
                    elif clean == 1:
                        start_index = text.find('general business overview')
                        text = text[start_index:]
                        end_index = text.find('unresolved staff comments')
                        text = text[:end_index]
                        #print('완료 : ', url, len(text))
                        key = 1
                    else:
                        pattern = r".*?i{0,2}tems?\s?(no.\s?)?(1|i|(1 i))?\s?(1a\s)?(2|1a|and\s2\s?)?(.\s?)?(discussion of|description of|our)?\s?(the\s?)?busines"
                        match = re.search(pattern, text)
                        if match:
                            start_index = match.end()
                            first_met = text.find("unresolved staff comments") #처음으로 만나는 끝 직전
                            if first_met == -1:
                                first_met = text.find("unresolved sec staff comments")
                                if first_met== -1:
                                    first_met = text.find("item 2 properties") 
                                    if first_met == -1:
                                        first_met = text.find("legal proceedings") 
                            if ((start_index - first_met) < 0) and ((start_index - first_met) > -1000): #목차에서 서로 나타난다는 뜻임 unresolved 항목은 어떻게든 반드시 목차에선 저 문구대로 나올 예정
                                #슬라이스 후 item1 business 찾고 unresovlved 그 이후 찾기
                                text = text[start_index+len('item'):]
                                match = re.search(pattern, text)
                                if match:
                                    start_index = match.end()
                                    text = text[start_index+len('item'):]
                                    end_index = text[10000:].find("unresolved staff comments") #처음으로 만나는 끝 직전
                                    if end_index == -1:
                                        end_index = text[10000:].find("unresolved sec staff comments")
                                        if end_index== -1:
                                            end_index = text[10000:].find("item 2 properties") 
                                            if end_index == -1:
                                                end_index = text[10000:].find("legal proceedings") 
                                    text=text[:end_index+10000]
                                    #print('완료 : ', url, len(text))
                                    key = 1
                            elif ((start_index - first_met) > 0) or ((start_index - first_met) < -1000): #목차에서 item1 business 형태가 아니라는 뜻임 아마 바로 항목으로 갈꺼임 / -1000은 목차가 없는 경우 방지 위해서 즉 바로 item1 나오는거
                                #슬라이스 후 바로 unresolved 찾기
                                text = text[start_index+len('item'):]
                                end_index = text[10000:].find("unresolved staff comments") #처음으로 만나는 끝 직전
                                if end_index == -1:
                                    end_index = text[10000:].find("unresolved sec staff comments")
                                    if end_index== -1:
                                        end_index = text[10000:].find("item 2 properties") 
                                        if end_index == -1:
                                            end_index = text[10000:].find("legal proceedings") 
                                text=text[:end_index+10000]
                                #print('완료 : ', url, len(text))
                                key = 1
                        else:
                            print('error2 : ', url)

                    #설치는 이미 했으면 필요는 없음
                    #nltk.download('punkt')
                    #nltk.download('averaged_perceptron_tagger')


                    if(key == 1):
                        text = text.replace('u.s.', 'us').replace('e.g.', 'eg')
                        sentences = text.lower().split('.')

                        #혹시 모르니 정제한 txt 파일을 저장을 해 놓자
                        #text_str = ''.join(text)
                        with open("C:/Users/이세준/자산평가/sentence/"+str(name)+"-"+str(fy)+".txt", 'w', encoding='utf-8') as file:
                            for sentence in sentences:
                                file.write(sentence.strip() + '\n') 
                            file.close()
                    else:
                        print('error3', url)

            else: 
                print('통신에러')
    except:
        logging.error(traceback.format_exc())
        

In [ ]:
import os
import multiprocessing
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, TimeoutError
import pandas as pd

def get_physical_cores():
    if os.name == 'posix':  # For Linux and macOS
        return int(os.environ.get('OMP_NUM_THREADS', multiprocessing.cpu_count() // 2))
    elif os.name == 'nt':  # For Windows
        return int(os.environ.get('NUMBER_OF_PROCESSORS', multiprocessing.cpu_count() // 2))
    else:
        return multiprocessing.cpu_count() // 2

physical_cores = get_physical_cores()

# 적절한 스레드 개수를 선택하세요 (예: CPU 코어 수)
num_threads = physical_cores//2
print("Using threads:", num_threads)

#ranges = [(i, i + 100) for i in range(11800, 15000, 100)] + [(15000, 15966)]
ranges = [(i, i + 100) for i in range(0, 15000, 1000)] + [(15000, 15966)]

result_data = pd.DataFrame()

#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained('bert-base-uncased')

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    executor.map(lambda x: textmaking(*x), ranges)
    #results = executor.map(lambda x: textmaking(*x), ranges)
    #for group_data_s in results:
        #result_data = pd.concat([result_data, group_data_s])
        
#result_data.to_csv('resutl.csv')

### 텍스트 데이터에서 grams 설정

In [ ]:
import os
#os.path.dirname(os.path.realpath(__file__))
dir = "C:/Users/이세준/자산평가/sentence/"
files = os.listdir(dir)
i = 0

with open(dir+files[i], 'r', encoding='utf-8') as file:
    file_content = file.read()
firmname = files[i].split('-')[0]
year = files[i].split('-')[1]

file_content = file_content.split('\n') #\n 형태로 메모장에 기록되어 있으므로 나누어줌
tokenList = [sent.split() for sent in file_content]
tokenList[:5] #문장을 \n으로 나누어서 생성, 앞 5개만 예시로

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import os


#처음 한번만 실행
#nltk.download('punkt')
#nltk.download('stopwords')

#stop word
stop_words = set(stopwords.words('english'))

# Porter Stemmer
stemmer = PorterStemmer()

processed_stemmed_list = []
for sentence in tokenList:
    processed_sentence = []
    for word in sentence:
        if word in stop_words or re.match(r'\d+', word): #stop_word나 숫자를 _로 변경
            word = '_'
        # Porter Stemmer 적용
        stemmed_word = stemmer.stem(word)
        processed_sentence.append(stemmed_word)
    processed_stemmed_list.append(processed_sentence)

print('기존 각 문장의 형태')
print(tokenList[:5])
print('머신러닝 위한 처리 후 문장의 형태')
print(processed_stemmed_list[:5])

from nltk.util import bigrams, trigrams

unigrams = [word for sentence in processed_stemmed_list for word in sentence if word != '_']

# Bigrams 생성 (_는 제거, 포함 안함)
bigrams_list = [bg for sentence in processed_stemmed_list for bg in bigrams(sentence) if '_' not in bg]

# Trigrams 생성 (_는 제거, 포함 안함) -> 트라이그램은 거의 없는거 같으니 제외한다.
#trigrams_list = [tg for sentence in processed_stemmed_list[:5] for tg in trigrams(sentence) if '_' not in tg]

print('unigrams 앞 5개 예시')
print(unigrams[:5])

print('biigrams 앞 5개 예시')
print(bigrams_list[:5])

from collections import Counter
uni_count = Counter(unigrams)
bi_count = Counter(bigrams_list)

### 예시로 10개만 추출하여서 빈도수 낮은거 없애는 연습

In [ ]:
dir = "C:/Users/이세준/자산평가/sentence/"
files = os.listdir(dir)

firm_year_gram_count = {}
for i in range(0, 10):#len(files)):
    with open(dir+files[i], 'r', encoding='utf-8') as file:
        file_content = file.read()
    firmname = files[i].split('-')[0]
    year = files[i].split('-')[1]

    file_content = file_content.split('\n') #\n 형태로 메모장에 기록되어 있으므로 나누어줌
    tokenList = [sent.split() for sent in file_content]

    #stop word
    stop_words = set(stopwords.words('english'))

    # Porter Stemmer
    stemmer = PorterStemmer()

    processed_stemmed_list = []
    for sentence in tokenList:
        processed_sentence = []
        for word in sentence:
            if word in stop_words or re.match(r'\d+', word): #stop_word나 숫자를 _로 변경
                word = '_'
            # Porter Stemmer 적용
            stemmed_word = stemmer.stem(word)
            processed_sentence.append(stemmed_word)
        processed_stemmed_list.append(processed_sentence)

    unigrams = [word for sentence in processed_stemmed_list for word in sentence if word != '_']
    uni_count = Counter(unigrams)

    bigrams_list = [bg for sentence in processed_stemmed_list for bg in bigrams(sentence) if '_' not in bg]
    bi_count = Counter(bigrams_list)
    
    key = f"{firmname}-{year}"
    firm_year_gram_count[key] = {'unigrams': uni_count, 'bigrams': bi_count}
    
#dictionary 형태로 firm_name과 year로 결합됨

### 10세트만 추출했으므로 3세트에 포함 안되는거 제거하는 방식 취해봄

In [ ]:
from collections import defaultdict

# 각 n-gram이 등장하는 세트 수를 세기 위한 dictionary
ngram_appearance_sets = defaultdict(set)

# 각 n-gram이 등장하는 세트 수를 셈
for key in firm_year_gram_count:
    for ngram in firm_year_gram_count[key]['unigrams']:
        ngram_appearance_sets[ngram].add(key)
    for ngram in firm_year_gram_count[key]['bigrams']:
        ngram_appearance_sets[ngram].add(key)

print('예시로 flower 가 나오는 세트수')
print(ngram_appearance_sets['flower'])
        
# 3개 세트 이상 ngram 들만 key값을 추춮
filtered_ngrams = {ngram for ngram, sets in ngram_appearance_sets.items() if len(sets) >= 3}

# firm_year_gram_count 내에서 추출된거만 따로 빼넴
for key in firm_year_gram_count:
    firm_year_gram_count[key]['unigrams'] = Counter({ngram: count for ngram, count in firm_year_gram_count[key]['unigrams'].items() if ngram in filtered_ngrams})
    firm_year_gram_count[key]['bigrams'] = Counter({ngram: count for ngram, count in firm_year_gram_count[key]['bigrams'].items() if ngram in filtered_ngrams})



In [ ]:
infrequent_ngrams = {ngram: sets for ngram, sets in ngram_appearance_sets.items() if len(sets) <= 3}

# 결과 출력
print('추가 예시로 3세트 밑에서 나오는 unigram 하나만 보면 ')
first_key = list(infrequent_ngrams.keys())[0]
print('단어 : ', first_key)
print(ngram_appearance_sets[first_key])

## 전체 수집한 txt 파일을 바탕으로 전체 세트에서 10개 미만으로 나타나는 gram들은 제외한다

In [ ]:
import pandas as pd

dir = "C:/Users/이세준/자산평가/sentence/"
files = os.listdir(dir)

firm_year_gram_count = {}
sample = pd.DataFrame()

for i in range(0, len(files)):
    with open(dir+files[i], 'r', encoding='utf-8') as file:
        file_content = file.read()
    file_name = files[i]
    # 연도 추출 - 문자 사이의 4자리 숫자임
    year_match = re.search(r'\b\d{4}\b', file_name)
    year = year_match.group() if year_match else None
    # 회사 이름 추출 (연도 바로 앞의 - 이전이 전부 회사이름임)
    if year:
        firmname = file_name.split(year)[0].rstrip('-')
    else:
        firmname = None
    sample.loc[i, 'name'] = firmname
    sample.loc[i, 'year'] = year

    file_content = file_content.split('\n') #\n 형태로 메모장에 기록되어 있으므로 나누어줌
    tokenList = [sent.split() for sent in file_content]

    #stop word
    stop_words = set(stopwords.words('english'))

    # Porter Stemmer
    stemmer = PorterStemmer()

    processed_stemmed_list = []
    for sentence in tokenList:
        processed_sentence = []
        for word in sentence:
            if word in stop_words or re.match(r'\d+', word): #stop_word나 숫자를 _로 변경
                word = '_'
            # Porter Stemmer 적용
            stemmed_word = stemmer.stem(word)
            processed_sentence.append(stemmed_word)
        processed_stemmed_list.append(processed_sentence)

    unigrams = [word for sentence in processed_stemmed_list for word in sentence if word != '_']
    uni_count = Counter(unigrams)

    bigrams_list = [bg for sentence in processed_stemmed_list for bg in bigrams(sentence) if '_' not in bg]
    bi_count = Counter(bigrams_list)
    
    key = f"{firmname}-{year}"
    firm_year_gram_count[key] = {'unigrams': uni_count, 'bigrams': bi_count}
    
#dictionary 형태로 firm_name과 year로 결합됨

ngram_appearance_sets = defaultdict(set)

# 각 n-gram이 등장하는 세트 수를 셈
for key in firm_year_gram_count:
    for ngram in firm_year_gram_count[key]['unigrams']:
        ngram_appearance_sets[ngram].add(key)
    for ngram in firm_year_gram_count[key]['bigrams']:
        ngram_appearance_sets[ngram].add(key)
        
# 10개 세트 이상 ngram 들만 key값을 추춮
filtered_ngrams = {ngram for ngram, sets in ngram_appearance_sets.items() if len(sets) >= 10}

# firm_year_gram_count 내에서 추출된거만 따로 빼넴
for key in firm_year_gram_count:
    firm_year_gram_count[key]['unigrams'] = Counter({ngram: count for ngram, count in firm_year_gram_count[key]['unigrams'].items() if ngram in filtered_ngrams})
    firm_year_gram_count[key]['bigrams'] = Counter({ngram: count for ngram, count in firm_year_gram_count[key]['bigrams'].items() if ngram in filtered_ngrams})



In [ ]:
#애써 만든 dictionary 시간관계상 나중에 써야되면 불러오려고
import pickle

sample.to_csv('gram_sample.csv', index=False, encoding='cp949')

# firm_year_gram_count를 파일로 저장
with open('firm_year_gram_count.pkl', 'wb') as f:
    pickle.dump(firm_year_gram_count, f)
    
with open('firm_year_gram_count.pkl', 'rb') as f:
    firm_year_gram_count = pickle.load(f)

### CAR 등 위해서 wrds 에서 결합하고자 cik 합침

In [ ]:
sample = pd.read_csv('gram_sample.csv', encoding='cp949')
alldf = pd.read_csv('ALL_df.csv', encoding='cp949')
alldf = alldf[['CIK', 'NAME']]
sample = sample.merge(alldf, left_on='name', right_on='NAME', how='left')
sample = sample.drop(columns='NAME')
sample = sample.drop_duplicates(subset=['name', 'year', 'CIK'])
sample.to_csv('gram_sample_CIK.csv', index=False, encoding='cp949')

### 없는거 몇개는 수기로 변경하고, 이후 wrds로 부터 주가, sec api를 이용하여 가진 10-K의 공시일을 받아옴

In [ ]:
ss = pd.read_csv('gram_sample_CIK.csv', encoding='cp949')

import requests

for i in range(0, len(ss)):
    cik = ss['CIK'][i]
    year = ss['year'][i]
    CIK = str(cik).zfill(10)
    url = 'https://data.sec.gov/api/xbrl/companyfacts/CIK'+CIK+'.json' #cik를 획득 하여 https://www.sec.gov/Archives/edgar/data/0000320193/0001193125-09-214859.txt 와 같이 cik-10k 형식 추출 예정
    headers = {'User-Agent': 'Mozilla'}
    res = requests.get(url, headers=headers)

    if res.status_code == 200:
        data = res.json()

        # facts 항목 가져오기
        facts = data.get('facts', {})
        if len(facts) >0:
            key1 = list(facts.keys())[0]
            key2 = list(facts[key1].keys())[0]
            key3 = list(facts[key1][key2]['units'].keys())[0]
            it =facts[key1][key2]['units'][key3]
            selected_items = []
            for item in it:
                if item.get('form') == '10-K' and item.get('fy') == year :
                    filed = item.get('filed', '')
                    ss.loc[i,'filed'] = filed #공시날짜를 받기 위해서
ss.to_csv('gram_sample_CIK_filed.csv', index=False, encoding='cp949')

In [ ]:
ss.head() #filed 가 공시 날짜

In [ ]:
import numpy as np
pr = pd.read_csv('gram_retrun.csv', encoding='cp949', low_memory=False)
pr = pr.drop(columns=['gvkey', 'iid'])
pr['diff'] = pr.groupby('cik', group_keys=False)['prccd'].apply(lambda x: np.log(x / x.shift(1))) #cik별 로그수익률 계산
#CAR을 위해서 AR을 capm 등이 아닌(각 회사별로 market index가 너무 제각각이라) 이전 250일 평균으로 간단하게 해봄
pr['rolling_avg'] = pr.groupby('cik')['diff'].transform(lambda x: x.rolling(window=250, min_periods=250).mean()) #이전 250일 평균
pr['AR'] = pr['diff'] - pr['rolling_avg'] #AR = r - E(r)
pr = pr.drop(columns=['rolling_avg'])

In [ ]:
ss = pd.read_csv('gram_sample_CIK_filed.csv', encoding='cp949')
for i in range(0, len(ss)):
    key = ss.loc[i, 'name']+'-'+str(ss.loc[i, 'year']) #우리가 사용할 회사명-year 형태의 key 값
    cik = ss.loc[i, 'CIK']
    filed = ss.loc[i, 'filed']
    filtered_rows = pr[(pr['cik'] == cik) & (pr['datadate'] >= filed)].copy() #cik가 만족하고, filed 즉 공시 이후 날짜를 기준으로 
    filtered_rows['datadate'] = pd.to_datetime(filtered_rows['datadate'])#nsmallest 를 쓰기 위해서 pd.datetime으로 바꾸고
    selected_rows = filtered_rows.nsmallest(2, 'datadate') #가장 날짜 빠른 두 날짜를 기준으로
    ar_sum = selected_rows['AR'].sum() #AR의 합을 구하고
    firm_year_gram_count[key]['CAR[0,1]'] = ar_sum #그걸 dictionary에 넣어놓음

In [ ]:
#CAR 구한거 까지 pickle로 합쳐줌 -> 혹시 이후에 다시 할 수도 있으니...
# firm_year_gram_count를 파일로 저장
with open('firm_year_gram_count_car.pkl', 'wb') as f:
    pickle.dump(firm_year_gram_count, f)
    


<io>
    SVM 부터는 여기서부터, 후에 회귀 분석시 머신러닝 뿐만 아니라 대조군 느낌으로 LM 긍정 부정 단어집도 사용할 예정임

In [ ]:
import pickle
import pandas as pd
#다시 pickle 불러옴
with open('firm_year_gram_count_car.pkl', 'rb') as f:
    firm_year_gram_count = pickle.load(f)
    
ss = pd.read_csv('gram_sample_CIK_filed.csv', encoding='cp949')


In [ ]:
# Loughran-McDonald Master Dictionary 기반, 긍정, 부정 단어  추출
LM = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv', encoding='cp949')
LM['Word'] = LM['Word'].str.lower()
LM = LM[['Word', 'Negative', 'Positive']]
LM['Negative'] = (LM['Negative'] != 0).astype(int)
LM['Positive'] = (LM['Positive'] != 0).astype(int)


In [ ]:
import pandas as pd
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

LM['Stemmed_Word'] = LM['Word'].apply(lambda x: stemmer.stem(str(x)) if pd.notnull(x) else x)
LM = LM[['Stemmed_Word', 'Negative', 'Positive']]
LM = LM.drop_duplicates()
LM.to_csv('LM.csv', index=False) #LM단어들 어근 (stem 기준으로 긍정, 부정 나누어봄)

In [ ]:
for i in range(0, len(ss)):
    ss.loc[i,'key'] = ss.loc[i, 'name']+'-'+str(ss.loc[i, 'year'])

In [ ]:
ss.head() #key 값도 같이 넣어서 이후에 다루기 편하도록

### dictionary에 LM 긍정, 부정 단어 비율 추가해봄

In [ ]:
import pickle
import pandas as pd

# 긍정, 부정 단어를 위에서 정제한 LM데이터프레임에서 리스트로 추출함
positive_words = LM[LM['Positive'] == 1]['Stemmed_Word'].tolist()
negative_words = LM[LM['Negative'] == 1]['Stemmed_Word'].tolist()

# pickle 파일에서 데이터 로드 (여기서부터 할 수도 있어서 위에서 불러왔지만 다시 불러옴)
with open('firm_year_gram_count_car.pkl', 'rb') as f:
    firm_year_gram_count = pickle.load(f)
con = pd.read_csv('gram_control.csv')
# 각 key에 대해 긍정 및 부정 단어 비율 계산
for key in firm_year_gram_count:
    pos_count = 0
    neg_count = 0
    total_count = 0

    # unigram 즉 공시 보고서에서 얼마나 긍정,부정단어 비율을 차지하는지를 고려해ㅗㅁ
    for word in firm_year_gram_count[key]['unigrams']:
        if word in positive_words:
            pos_count += 1
        elif word in negative_words:
            neg_count += 1
        total_count += 1

    if total_count > 0:
        firm_year_gram_count[key]['positive_ratio'] = pos_count / total_count
        firm_year_gram_count[key]['negative_ratio'] = neg_count / total_count
        firm_year_gram_count[key]['PN'] = (pos_count - neg_count) / (pos_count + neg_count) #2011 price 방식
        firm_year_gram_count[key]['PN2'] = (pos_count - neg_count) / total_count #전체 대비
    else:
        firm_year_gram_count[key]['positive_ratio'] = 0
        firm_year_gram_count[key]['negative_ratio'] = 0
        firm_year_gram_count[key]['PN'] = 0
        firm_year_gram_count[key]['PN2'] = 0

    #통제변수도 같이 넣어줌
    def get_value(df, cik, year, column): #값이 없는 경우 대비
        filtered_df = df[(df['cik'] == cik) & (df['fyear'] == year)]
        if not filtered_df.empty:
            return filtered_df[column].iloc[0]
        return None
    
    cik = ss[ss['key']==key]['CIK'].iloc[0]
    year = ss[ss['key']==key]['year'].iloc[0]
    bperhare = get_value(con, cik, year, 'bpershare')
    sale = get_value(con, cik, year, 'sale')
    lnmkv = get_value(con, cik, year, 'ln(mkv)')
    firm_year_gram_count[key]['sale'] = sale
    firm_year_gram_count[key]['lnmkv'] = lnmkv
    firm_year_gram_count[key]['bperhare'] = bperhare



#긍정, 부정 비율까지도 합쳐줌.. 혹시 다시 쓸수도 있으니
with open('firm_year_gram_count_car_LMratio.pkl', 'wb') as f:
    pickle.dump(firm_year_gram_count, f)

In [1]:
import pickle
import pandas as pd
#다시 pickle 불러옴
with open('firm_year_gram_count_car_LMratio.pkl', 'rb') as f:
    firm_year_gram_count = pickle.load(f)
    
ss = pd.read_csv('gram_sample_CIK_filed.csv', encoding='cp949')
for i in range(0, len(ss)):
    ss.loc[i,'key'] = ss.loc[i, 'name']+'-'+str(ss.loc[i, 'year'])

import cupy as cp
# 전체 unigrams와 bigrams 집합 생성
all_unigrams = set()
all_bigrams = set()
for firm in firm_year_gram_count:
    all_unigrams.update(firm_year_gram_count[firm]['unigrams'].keys())
    all_bigrams.update(firm_year_gram_count[firm]['bigrams'].keys())

# 전체 어휘 집합 생성
vocabulary = list(all_unigrams) + ['_'.join(bigram) for bigram in all_bigrams]

# 판다스(cpu)사용 vocal 이용 단어 count된 데이터프레임만들기

In [ ]:
import pandas as pd
#ii=len(df)
batch_size = 1000

total_firms = le(ss)
# 열 이름을 숫자 형태로 지정
column_names = list(range(len(vocabulary)))


for k in range(0, total_firms, batch_size):
    # 데이터프레임 초기화
    df = pd.DataFrame(columns=column_names)

    # 각 key에 대한 데이터 처리
    for i in range(k, min(k + batch_size, total_firms)):
        firm = ss.loc[i, 'key']
        # 각 단어의 빈도를 저장할 리스트
        firm_vector = [0] * len(vocabulary)
        for j, word in enumerate(vocabulary):
            if word in firm_year_gram_count[firm]['unigrams']:
                firm_vector[j] = firm_year_gram_count[firm]['unigrams'][word]
            elif word in firm_year_gram_count[firm]['bigrams']:
                firm_vector[j] = firm_year_gram_count[firm]['bigrams'][word]
    
        # 데이터프레임에 행 추가
        df2 = pd.DataFrame([firm_vector], columns=column_names)
        df = pd.concat([df, df2], ignore_index=True)
    df.to_pickle(str(i)+".pkl")


### 14105개에 대해서 최대 500개 까지가 현재 사양에서 돌아가는 한계라서 455개씩 배치형태를 이용하여 SVR해보기로함

In [2]:
import pandas as pd
import pickle
import cuml
import cudf
import gc
import time
import numpy as np

ss = pd.read_csv('gram_sample_CIK_filed.csv', encoding='cp949')
for i in range(0, len(ss)):
    ss.loc[i,'key'] = ss.loc[i, 'name']+'-'+str(ss.loc[i, 'year'])

with open('firm_year_gram_count_car_LMratio.pkl', 'rb') as f:
    firm_year_gram_count = pickle.load(f)

### SVR기반 CAR 예측

In [2]:
from sklearn.metrics import mean_squared_error

#svr_model 구축
svr_model = cuml.SVR(kernel='rbf')

batch = 455
end = len(ss)
total_time = 0
mse_values = []

for batch_start in range(0, end, batch):
    batch_end = min(batch_start + batch, end)
    pk = pd.read_pickle(str(batch_end)+"_1.pkl") 
    pk = pk.astype('float32')
    pk_cudf = cudf.DataFrame.from_pandas(pk)
    
    
    y = []
    for i in range(batch_start, batch_end):
        firm = ss.loc[i, 'key']
        y.append(firm_year_gram_count[firm]['CAR[0,1]'])
    y = pd.DataFrame(y, columns=['CAR'])
    y_cudf = cudf.Series(y['CAR'])    

    #머신러닝 자체의 시간을 계산해봄
    start_time = time.time() 
    svr_model.fit(pk_cudf, y_cudf)
    predictions = svr_model.predict(pk_cudf)
    predictions_np = predictions.to_numpy()
    mse = mean_squared_error(y['CAR'], predictions_np) #mse 계산
    mse_values.append(mse)
    
    svr=pd.DataFrame(predictions_np, columns=['SVR'])
    svr = pd.concat([y, svr], axis=1)
    svr.to_csv(str(batch_end)+'_SVR_GPU.csv', encoding='cp949') #svr.to_csv(str(batch_start)+'_'+str(batch_end)+'.csv', encoding='cp949') 
    end_time = time.time()
    # 총 걸린시간에 합산함
    total_time += (end_time - start_time)
    
    del pk, pk_cudf, y, y_cudf ,svr, predictions, predictions_np
    gc.collect()  

print('총 SVR 학습 시간 (GPU):', total_time)
overall_mse = np.mean(mse_values)
overall_rmse = np.sqrt(overall_mse)
print('전체 RMSE:', overall_rmse)

del svr_model
gc.collect()

총 SVR 학습 시간 (GPU): 2208.715969324112
전체 RMSE: 0.1146646217346256


0

### CPU SVR

In [5]:
from sklearn.svm import SVR
import pandas as pd
import time

# CPU 기반 SVR 모델 구축
svr_model = SVR(kernel='rbf')

batch = 455
end = len(ss)
total_time = 0
mse_values = []

for batch_start in range(0, end, batch):
    batch_end = min(batch_start + batch, end)
    pk = pd.read_pickle(str(batch_end) + "_1.pkl")
    pk = pk.astype('float32')
    
    y = []
    for i in range(batch_start, batch_end):
        firm = ss.loc[i, 'key']
        y.append(firm_year_gram_count[firm]['CAR[0,1]'])
    y = pd.DataFrame(y, columns=['CAR'])

    #머신러닝 자체의 시간을 계산해봄
    start_time = time.time() 
    svr_model.fit(pk, y['CAR'])
    predictions = svr_model.predict(pk)
    mse = mean_squared_error(y['CAR'], predictions) #mse 계산
    mse_values.append(mse)
    
    # 결과 저장
    svr_results = pd.DataFrame(predictions, columns=['SVR'])
    svr_results = pd.concat([y, svr_results], axis=1)
    svr_results.to_csv(str(batch_end) + '_SVR_CPU.csv', encoding='cp949')
    end_time = time.time()
    # 총 걸린시간에 합산함
    total_time += (end_time - start_time)
    
    # 메모리 정리
    del pk, y, svr_results, predictions
    gc.collect()

print('총 SVR 학습 시간 (CPU):', total_time)
overall_mse = np.mean(mse_values)
overall_rmse = np.sqrt(overall_mse)
print('전체 RMSE:', overall_rmse)

del svr_model
gc.collect()

총 SVR 학습 시간 (CPU): 546.223879814148
전체 RMSE: 0.11466489642813031


0

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import time
import gc
import numpy as np

# CPU 기반 Random Forest 모델 구축
rf_model = RandomForestRegressor()

batch = 455
end = len(ss)
total_time = 0
mse_values = []

for batch_start in range(0, end, batch):
    batch_end = min(batch_start + batch, end)
    pk = pd.read_pickle(str(batch_end) + "_1.pkl")
    pk = pk.astype('float32')
    
    y = []
    for i in range(batch_start, batch_end):
        firm = ss.loc[i, 'key']
        y.append(firm_year_gram_count[firm]['CAR[0,1]'])
    y = pd.DataFrame(y, columns=['CAR'])

    # 머신러닝 학습과 예측 시간 측정
    start_time = time.time() 
    rf_model.fit(pk, y['CAR'])
    predictions = rf_model.predict(pk)
    mse = mean_squared_error(y['CAR'], predictions) # MSE 계산
    mse_values.append(mse)
    
    # 결과 저장
    rf_results = pd.DataFrame(predictions, columns=['RF'])
    rf_results = pd.concat([y, rf_results], axis=1)
    rf_results.to_csv(str(batch_end) + '_RF_CPU.csv', encoding='cp949')
    end_time = time.time()
    # 총 걸린 시간에 합산
    total_time += (end_time - start_time)
    
    # 메모리 정리
    del pk, y, rf_results, predictions
    gc.collect()

print('총 Random Forest 학습 시간 (CPU):', total_time)
overall_mse = np.mean(mse_values)
overall_rmse = np.sqrt(overall_mse)
print('전체 RMSE:', overall_rmse)

del rf_model
gc.collect()


총 Random Forest 학습 시간 (CPU): 5095.119078159332
전체 RMSE: 0.045324268005599834


0

### 그래디언트부스트 이용

In [3]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import time
import gc
import numpy as np

# CPU 기반 Gradient Boosting 모델 구축
gb_model = GradientBoostingRegressor()

batch = 455
end = len(ss)
total_time = 0
mse_values = []

for batch_start in range(0, end, batch):
    batch_end = min(batch_start + batch, end)
    pk = pd.read_pickle(str(batch_end) + "_1.pkl")
    pk = pk.astype('float32')
    
    y = []
    for i in range(batch_start, batch_end):
        firm = ss.loc[i, 'key']
        y.append(firm_year_gram_count[firm]['CAR[0,1]'])
    y = pd.DataFrame(y, columns=['CAR'])

    # 머신러닝 학습과 예측 시간 측정
    start_time = time.time() 
    gb_model.fit(pk, y['CAR'])
    predictions = gb_model.predict(pk)
    mse = mean_squared_error(y['CAR'], predictions) # MSE 계산
    mse_values.append(mse)
    
    # 결과 저장
    gb_results = pd.DataFrame(predictions, columns=['GB'])
    gb_results = pd.concat([y, gb_results], axis=1)
    gb_results.to_csv(str(batch_end) + '_GB_CPU.csv', encoding='cp949')
    end_time = time.time()
    # 총 걸린 시간에 합산
    total_time += (end_time - start_time)
    
    # 메모리 정리
    del pk, y, gb_results, predictions
    gc.collect()

print('총 Gradient Boosting 학습 시간 (CPU):', total_time)
overall_mse = np.mean(mse_values)
overall_rmse = np.sqrt(overall_mse)
print('전체 RMSE:', overall_rmse)

del gb_model
gc.collect()


총 Gradient Boosting 학습 시간 (CPU): 1349.7038168907166
전체 RMSE: 0.0316019272017455


0

In [25]:
import pandas as pd

batch = 455
end = len(ss)
d1 = pd.DataFrame()
for batch_start in range(0, end, batch):
    batch_end = min(batch_start + batch, end)
    d2 = pd.read_csv(str(batch_end)+'_GB_CPU.csv',index_col = 0)
    d2 = d2.rename(columns={'GB': 'GB_CPU'})
    
    d3 = pd.read_csv(str(batch_end)+'_RF_CPU.csv',index_col = 0)
    d3 = d3.rename(columns={'RF': 'RF_CPU'})
    d3 = d3.drop(columns = ['CAR'])
    
    d4 = pd.read_csv(str(batch_end)+'_SVR_CPU.csv',index_col = 0)
    d4 = d4.rename(columns={'SVR': 'SVR_CPU'})
    d4 = d4.drop(columns = ['CAR'])

    d5 = pd.read_csv(str(batch_end)+'_SVR_GPU.csv',index_col = 0)
    d5 = d5.rename(columns={'SVR': 'SVR_GPU'})
    d5 = d5.drop(columns = ['CAR'])

    dd = pd.concat([d2, d3, d4, d5], axis=1)
    d1 = pd.concat([d1, dd], ignore_index=True)

d1.to_csv('CAR_455.csv', encoding='cp949')

<oi>
    배치의 크기는 455개로 작고, 각 데이터 (배치 내에서의 각 row들) 이 47만에 달하는 컬럼수를 가지는 탓에 CPU로 작업을 한 경우 더 빠른 것으로 나타나고 있다. <br>
    머신러닝 적합에 있어서 병렬 구조의 gpu는 cpu보다 적합에 있어 더 빠른 것으로 알려져 있으나, 이 데이터는 기형적으로 각 데이터의 컬럼수 가 너무 많은 탓에 <br>
    최대 불러올 수 있는 크기가 500개 언저리인 탓에 gpu의 효유성을 느끼지 못하고 있는 것 같다. <br>
    또한 GPU로 랜덤포레스트를 해보려고 하여도 트리의 깊이가 워낙 깊어 메모리 에러로 인해 작업을 수행할 수 없기에, 기형적인 이 데이터 셋에 대해서는 CPU 위주로 작업을 하고 <br>
    배치 사이즈를 더 키워서 머신러닝 적합을 해 보고자 한다. <br>

<center><b>머신러닝 결과 표 - 455개의 배치</b></center>

<br>

|       | Time  | RMSE | 
|-------|-----------------|--------------------|
| SVR(GPU)  | 2208   | 0.1146  | 
| SVR(CPU)  | 546 |  0.1146      |
| RF(CPU)  | 5095   | 0.0453      | 
| GB(CPU) | 1349   | 0.0316     |

### 455*31 개의 형태이므로 6개씩 묶어서 작업하고 마지막에 대해선 7개로 작업을 진행함

### CPU기반 SVR, 455배치 3개씩

### 데이터 다시 불러오기

In [1]:
import pandas as pd
import pickle
import cuml
import cudf
import gc
import time
import numpy as np

ss = pd.read_csv('gram_sample_CIK_filed.csv', encoding='cp949')
for i in range(0, len(ss)):
    ss.loc[i,'key'] = ss.loc[i, 'name']+'-'+str(ss.loc[i, 'year'])

with open('firm_year_gram_count_car_LMratio.pkl', 'rb') as f:
    firm_year_gram_count = pickle.load(f)

In [3]:
from sklearn.svm import SVR
import pandas as pd
import time
from sklearn.metrics import mean_squared_error
import numpy as np

# CPU 기반 SVR 모델 구축
svr_model = SVR(kernel='rbf')

batch = 455*3
end = len(ss) - 455
total_time = 0
mse_values = []

for batch_start in range(0, end, batch):
    y = []
    
    batch_end1 = min(batch_start + 455, end)
    pk1 = pd.read_pickle(str(batch_end1) + "_1.pkl")
    pk1 = pk1.astype('float32')
        
    batch_end2 = min(batch_end1 + 455, end)
    pk2 = pd.read_pickle(str(batch_end2) + "_1.pkl")
    pk2 = pk2.astype('float32')
    
    batch_end3 = min(batch_end2 + 455, end)
    pk3 = pd.read_pickle(str(batch_end3) + "_1.pkl")
    pk3 = pk3.astype('float32')
   
    for i in range(batch_start, batch_end3):
        firm = ss.loc[i, 'key']
        y.append(firm_year_gram_count[firm]['CAR[0,1]'])
    
    if batch_end3 == end:
        pk4 = pd.read_pickle(str(batch_end3+455) + "_1.pkl")
        pk4 = pk4.astype('float32')
        pk3 = pd.concat([pk3, pk4], ignore_index=True)

        for i in range(batch_end3, batch_end3+455):
            firm = ss.loc[i, 'key']
            y.append(firm_year_gram_count[firm]['CAR[0,1]'])
       
    pk = pd.concat([pk1,pk2,pk3])  
    y = pd.DataFrame(y, columns=['CAR'])

    #머신러닝 자체의 시간을 계산해봄
    start_time = time.time() 
    svr_model.fit(pk, y['CAR'])
    predictions = svr_model.predict(pk)
    mse = mean_squared_error(y['CAR'], predictions) #mse 계산
    mse_values.append(mse)
    
    # 결과 저장
    svr_results = pd.DataFrame(predictions, columns=['SVR'])
    svr_results = pd.concat([y, svr_results], axis=1)
    svr_results.to_csv(str(batch_end3) + '_SVR_CPU_4556.csv', encoding='cp949')
    end_time = time.time()
    # 총 걸린시간에 합산함
    total_time += (end_time - start_time)
    
    # 메모리 정리
    del pk, y, svr_results, predictions
    gc.collect()

print('총 SVR 학습 시간 (CPU):', total_time)
overall_mse = np.mean(mse_values)
overall_rmse = np.sqrt(overall_mse)
print('전체 RMSE:', overall_rmse)

del svr_model
gc.collect()

총 SVR 학습 시간 (CPU): 1305.7770247459412
전체 RMSE: 0.11857238968542118


0

### CPU기반 RF, 455배치 3개씩

In [2]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import time
from sklearn.metrics import mean_squared_error
import numpy as np

# CPU 기반 Random Forest 모델 구축
rf_model = RandomForestRegressor()

batch = 455*3
end = len(ss) - 455
total_time = 0
mse_values = []

for batch_start in range(0, end, batch):
    y = []
    
    batch_end1 = min(batch_start + 455, end)
    pk1 = pd.read_pickle(str(batch_end1) + "_1.pkl")
    pk1 = pk1.astype('float32')
        
    batch_end2 = min(batch_end1 + 455, end)
    pk2 = pd.read_pickle(str(batch_end2) + "_1.pkl")
    pk2 = pk2.astype('float32')
    
    batch_end3 = min(batch_end2 + 455, end)
    pk3 = pd.read_pickle(str(batch_end3) + "_1.pkl")
    pk3 = pk3.astype('float32')
   
    for i in range(batch_start, batch_end3):
        firm = ss.loc[i, 'key']
        y.append(firm_year_gram_count[firm]['CAR[0,1]'])
    
    if batch_end3 == end:
        pk4 = pd.read_pickle(str(batch_end3+455) + "_1.pkl")
        pk4 = pk4.astype('float32')
        pk3 = pd.concat([pk3, pk4], ignore_index=True)

        for i in range(batch_end3, batch_end3+455):
            firm = ss.loc[i, 'key']
            y.append(firm_year_gram_count[firm]['CAR[0,1]'])
       
    pk = pd.concat([pk1,pk2,pk3])  
    y = pd.DataFrame(y, columns=['CAR'])

    # 머신러닝 학습과 예측 시간 측정
    start_time = time.time() 
    rf_model.fit(pk, y['CAR'])
    predictions = rf_model.predict(pk)
    mse = mean_squared_error(y['CAR'], predictions) # MSE 계산
    mse_values.append(mse)
    
    # 결과 저장
    rf_results = pd.DataFrame(predictions, columns=['RF'])
    rf_results = pd.concat([y, rf_results], axis=1)
    rf_results.to_csv(str(batch_end3) + '_RF_CPU_4556.csv', encoding='cp949')
    end_time = time.time()
    # 총 걸린 시간에 합산
    total_time += (end_time - start_time)
    
    # 메모리 정리
    del pk, y, rf_results, predictions
    gc.collect()

print('총 Random Forest 학습 시간 (CPU):', total_time)
overall_mse = np.mean(mse_values)
overall_rmse = np.sqrt(overall_mse)
print('전체 RMSE:', overall_rmse)

del rf_model
gc.collect()

총 Random Forest 학습 시간 (CPU): 8101.543528556824
전체 RMSE: 0.05139839379391767


0

### CPU기반 GB, 455배치 3개씩

In [3]:
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
import time
from sklearn.metrics import mean_squared_error
import numpy as np

# CPU 기반 Gradient Boosting 모델 구축
gb_model = GradientBoostingRegressor()

batch = 455*3
end = len(ss) - 455
total_time = 0
mse_values = []

for batch_start in range(0, end, batch):
    y = []
    
    batch_end1 = min(batch_start + 455, end)
    pk1 = pd.read_pickle(str(batch_end1) + "_1.pkl")
    pk1 = pk1.astype('float32')
        
    batch_end2 = min(batch_end1 + 455, end)
    pk2 = pd.read_pickle(str(batch_end2) + "_1.pkl")
    pk2 = pk2.astype('float32')
    
    batch_end3 = min(batch_end2 + 455, end)
    pk3 = pd.read_pickle(str(batch_end3) + "_1.pkl")
    pk3 = pk3.astype('float32')
   
    for i in range(batch_start, batch_end3):
        firm = ss.loc[i, 'key']
        y.append(firm_year_gram_count[firm]['CAR[0,1]'])
    
    if batch_end3 == end:
        pk4 = pd.read_pickle(str(batch_end3+455) + "_1.pkl")
        pk4 = pk4.astype('float32')
        pk3 = pd.concat([pk3, pk4], ignore_index=True)

        for i in range(batch_end3, batch_end3+455):
            firm = ss.loc[i, 'key']
            y.append(firm_year_gram_count[firm]['CAR[0,1]'])
       
    pk = pd.concat([pk1,pk2,pk3])  
    y = pd.DataFrame(y, columns=['CAR'])

    # 머신러닝 학습과 예측 시간 측정
    start_time = time.time() 
    gb_model.fit(pk, y['CAR'])
    predictions = gb_model.predict(pk)
    mse = mean_squared_error(y['CAR'], predictions) # MSE 계산
    mse_values.append(mse)
    
    # 결과 저장
    gb_results = pd.DataFrame(predictions, columns=['GB'])
    gb_results = pd.concat([y, gb_results], axis=1)
    gb_results.to_csv(str(batch_end3) + '_GB_CPU_4556.csv', encoding='cp949')
    end_time = time.time()
    # 총 걸린 시간에 합산
    total_time += (end_time - start_time)
    
    # 메모리 정리
    del pk, y, gb_results, predictions
    gc.collect()

print('총 Gradient Boosting 학습 시간 (CPU):', total_time)
overall_mse = np.mean(mse_values)
overall_rmse = np.sqrt(overall_mse)
print('전체 RMSE:', overall_rmse)

del gb_model
gc.collect()

총 Gradient Boosting 학습 시간 (CPU): 1225.5256366729736
전체 RMSE: 0.05017679497557742


0

In [15]:
import pandas as pd

batch = 455*3
end = len(ss) - 455
d = pd.read_csv('CAR_455.csv', encoding='cp949', index_col=0)
d1 = pd.DataFrame()
for batch_start in range(0, end, batch):
    batch_end = min(batch_start + batch, end)
    d2 = pd.read_csv(str(batch_end)+'_GB_CPU_4556.csv',index_col = 0)
    d2 = d2.rename(columns={'GB': 'GB_CPU_4553'})
    d2 = d2.drop(columns = ['CAR'])
    
    d3 = pd.read_csv(str(batch_end)+'_RF_CPU_4556.csv',index_col = 0)
    d3 = d3.rename(columns={'RF': 'RF_CPU_4553'})
    d3 = d3.drop(columns = ['CAR'])
    
    d4 = pd.read_csv(str(batch_end)+'_SVR_CPU_4556.csv',index_col = 0)
    d4 = d4.rename(columns={'SVR': 'SVR_CPU_4553'})
    d4 = d4.drop(columns = ['CAR'])

    dd = pd.concat([d2, d3, d4], axis=1)
    d1 = pd.concat([d1, dd], ignore_index=True)

d = pd.concat([d, d1], axis=1)
d.to_csv('CAR_4553.csv', encoding='cp949')

<center><b>머신러닝 결과 표 - 455*3 배치 추가</b></center>

<br>

|       | Time  | RMSE | 
|-------|-----------------|--------------------|
| SVR(GPU)  | 2208   | 0.1146  | 
| SVR(CPU)  | 546 |  0.1146      |
| RF(CPU)  | 5095   | 0.0453      | 
| GB(CPU) | 1349   | 0.0316     |
| SVR3(CPU)  | 1305 |  0.1185     |
| RF3(CPU)  | 8101  | 0.0513     | 
| GB3(CPU) | 1225   | 0.0517    |
</table>
<br>
그래디언트 부스트를 제외하고, 전부 작업 시간이 증가하였으며, 모든 모델 전부 RMSE가 증가한 것을 확인 가능하다. <br>
이제 통제변수를 추가하여, LM 단어집과, 머신러닝을 이용하여 예측한 CAR 값들 간의 회귀를 통하여 어떠한 결과가 나타나는지 확인해 보도록 하겠다 <br>
뿐만 아니라, 기존의 선행연구들과 유사하게, 기업 특성 통제변수를 몇 가지 추가한 상태로 회귀를 진행해보도록 하자 <br>

In [11]:
firm_year_gram_count['1 800 FLOWERS COM INC-2014'].keys()

dict_keys(['unigrams', 'bigrams', 'CAR[0,1]', 'positive_ratio', 'negative_ratio', 'PN', 'PN2', 'sale', 'lnmkv', 'bperhare'])

In [16]:
pos = []
neg = []
PN = []
PN2 = []
sale = []
lnmkv = []
bperhare = []
for i in range(0, len(ss)):
    firm = ss.loc[i, 'key'] #불러오는 이유는 혹시 dictionary에서 순서가 잘못되었을수도 있어서.. key 기준으로 기존 머신러닝 정렬했으니 통일하려고
    pos.append(firm_year_gram_count[firm]['positive_ratio'])
    neg.append(firm_year_gram_count[firm]['negative_ratio'])
    PN.append(firm_year_gram_count[firm]['PN'])
    PN2.append(firm_year_gram_count[firm]['PN2'])
    sale.append(firm_year_gram_count[firm]['sale'])
    lnmkv.append(firm_year_gram_count[firm]['lnmkv'])
    bperhare.append(firm_year_gram_count[firm]['bperhare'])
pos = pd.DataFrame(pos, columns=['positive_ratio'])
neg = pd.DataFrame(neg, columns=['negative_ratio'])
PN = pd.DataFrame(PN, columns=['PN'])
PN2 = pd.DataFrame(PN2, columns=['PN2'])
sale = pd.DataFrame(sale, columns=['sale'])
lnmkv = pd.DataFrame(lnmkv, columns=['lnmkv'])
bperhare = pd.DataFrame(bperhare, columns=['bperhare'])

ddd = pd.concat([d,pos, neg, PN, PN2, sale, lnmkv, bperhare], axis=1)

ddd.to_csv('CAR_control.csv', encoding='cp949')

In [24]:
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

X = ['GB_CPU', 'RF_CPU', 'SVR_CPU', 'SVR_GPU', 'GB_CPU_4553','RF_CPU_4553',
     'SVR_CPU_4553', 'positive_ratio', 'negative_ratio', 'PN', 'PN2']

models = []

for x in X:
    formula = f'CAR ~ sale + lnmkv + bperhare + {x}'
    model = smf.ols(formula, data=ddd).fit()
    models.append(model)
          
dfoutput = summary_col(models, stars=True, model_names=X)
print(dfoutput)


                 GB_CPU     RF_CPU    SVR_CPU   SVR_GPU  GB_CPU_4553 RF_CPU_4553 SVR_CPU_4553 positive_ratio negative_ratio     PN        PN2    
-------------------------------------------------------------------------------------------------------------------------------------------------
Intercept      -0.0016**  -0.0004    0.0095*** 0.0095*** -0.0030***  -0.0008     0.0117***    0.0111         0.0252***      0.0180***  0.0163*** 
               (0.0007)   (0.0007)   (0.0024)  (0.0024)  (0.0011)    (0.0008)    (0.0025)     (0.0077)       (0.0058)       (0.0043)   (0.0039)  
sale           -0.0000    -0.0000*   -0.0000   -0.0000   -0.0000     -0.0000     -0.0000      0.0000         0.0000         0.0000     0.0000    
               (0.0000)   (0.0000)   (0.0000)  (0.0000)  (0.0000)    (0.0000)    (0.0000)     (0.0000)       (0.0000)       (0.0000)   (0.0000)  
lnmkv          0.0003**   0.0001     -0.0003   -0.0003   0.0005***   0.0001      -0.0007*     -0.0013***     -0.0011***    

### result & conclusion

<io>
분석 결과, 455개의 배치 기준으로 각각에 머신러닝 적합을 한 경우, 3 배치씩 묶어서 머신러닝 적합을 한 결과에 비해 더 높은 R-square를 보였다. <br>
하지만 Randomforrest 의 경우, R_sqaure의 하락이 그렇게 크지 않았으며, 3 모형 중 랜덤포레스트가 가장 높은 R-square를 보였다. <br>
또한, 기존의 LM 단어집에 비해 머신러닝 적합을 한 경우 더 높은 R-squre를 보였으며, 긍정 단어의 수는 유의성이 나타나지 않았으며 <br>
부정 단어의 음의 유의성과, 긍정/부정 단어 비율의 경우 유의한 양의 결과가 나오는 것은 10-K filling date의 직후 CAR을 다룬 기존 선행연구들과 유사한 결과이다. <br>
허나, LM단어의 10-K 예측이 최근에 연구에선 잘 맞지 않다는 연구결과가 존재하기에, 지금의 방식과 같이 머신러닝 기반 예측이 도움이 될 것으로 기대된다. <br>
또한, 그 중에서 random forrest의 경우 예측에 잘 맞는 tree 만 쓴다는 점에서 프루닝의 개념이 첨가되어 있어, 더욱 더 높은 정확도를 보이는 것으로 예상된다. <br>

추후 연구에선, 현재 각 data 행의 경우 47만개의 row이기 때문에 gpu의 장점을 살리지 못하기에, 자연어 토큰 기반, 딥러닝을 첨가한 요약집을 바탕으로 열 수를 줄여 <br>
GPU와 CPU의 성능, 효율 등을 비교 가능할 것이며, 사업설명이 적혀있는 business description 부분만이 아닌 전체 데이터에 대한 분석 역시 진행 가능할 것 이다. <br>

### 추가 분석 : RF가 성능이 제일 좋다면 decision tree를 이용한다면?

In [25]:
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import time
from sklearn.metrics import mean_squared_error
import numpy as np
import gc

dt_model = DecisionTreeRegressor() #ccp_alpha=0.01 -> 프루닝시, 만일 DT 성능 종으면 프루닝 숫자 바꿔가면서 마지막으로 해보자

batch = 455*3
end = len(ss) - 455
total_time = 0
mse_values = []

for batch_start in range(0, end, batch):
    y = []
    
    batch_end1 = min(batch_start + 455, end)
    pk1 = pd.read_pickle(str(batch_end1) + "_1.pkl")
    pk1 = pk1.astype('float32')
        
    batch_end2 = min(batch_end1 + 455, end)
    pk2 = pd.read_pickle(str(batch_end2) + "_1.pkl")
    pk2 = pk2.astype('float32')
    
    batch_end3 = min(batch_end2 + 455, end)
    pk3 = pd.read_pickle(str(batch_end3) + "_1.pkl")
    pk3 = pk3.astype('float32')
   
    for i in range(batch_start, batch_end3):
        firm = ss.loc[i, 'key']
        y.append(firm_year_gram_count[firm]['CAR[0,1]'])
    
    if batch_end3 == end:
        pk4 = pd.read_pickle(str(batch_end3+455) + "_1.pkl")
        pk4 = pk4.astype('float32')
        pk3 = pd.concat([pk3, pk4], ignore_index=True)

        for i in range(batch_end3, batch_end3+455):
            firm = ss.loc[i, 'key']
            y.append(firm_year_gram_count[firm]['CAR[0,1]'])
       
    pk = pd.concat([pk1,pk2,pk3])  
    y = pd.DataFrame(y, columns=['CAR'])

    # 머신러닝 학습과 예측 시간 측정
    start_time = time.time()
    dt_model.fit(pk, y['CAR'])
    predictions = dt_model.predict(pk)
    mse = mean_squared_error(y['CAR'], predictions) # MSE 계산
    mse_values.append(mse)
    
    # 결과 저장
    dt_results = pd.DataFrame(predictions, columns=['DT'])
    dt_results = pd.concat([y, dt_results], axis=1)
    dt_results.to_csv(str(batch_end3) + '_DT_CPU_4556.csv', encoding='cp949')
    end_time = time.time()
    total_time += (end_time - start_time)
    
    # 메모리 정리
    del pk, y, dt_results, predictions
    gc.collect()

print('총 Decision Tree 학습 시간 (CPU):', total_time)
overall_mse = np.mean(mse_values)
overall_rmse = np.sqrt(overall_mse)
print('전체 RMSE:', overall_rmse)

del dt_model
gc.collect()


총 Decision Tree 학습 시간 (CPU): 191.86391687393188
전체 RMSE: 0.0010875659563371579


0

In [26]:
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import time
from sklearn.metrics import mean_squared_error
import numpy as np
import gc

dt_model = DecisionTreeRegressor(ccp_alpha=0.001) #ccp_alpha=0.01 -> 프루닝시, 만일 DT 성능 종으면 프루닝 숫자 바꿔가면서 마지막으로 해보자

batch = 455*3
end = len(ss) - 455
total_time = 0
mse_values = []

for batch_start in range(0, end, batch):
    y = []
    
    batch_end1 = min(batch_start + 455, end)
    pk1 = pd.read_pickle(str(batch_end1) + "_1.pkl")
    pk1 = pk1.astype('float32')
        
    batch_end2 = min(batch_end1 + 455, end)
    pk2 = pd.read_pickle(str(batch_end2) + "_1.pkl")
    pk2 = pk2.astype('float32')
    
    batch_end3 = min(batch_end2 + 455, end)
    pk3 = pd.read_pickle(str(batch_end3) + "_1.pkl")
    pk3 = pk3.astype('float32')
   
    for i in range(batch_start, batch_end3):
        firm = ss.loc[i, 'key']
        y.append(firm_year_gram_count[firm]['CAR[0,1]'])
    
    if batch_end3 == end:
        pk4 = pd.read_pickle(str(batch_end3+455) + "_1.pkl")
        pk4 = pk4.astype('float32')
        pk3 = pd.concat([pk3, pk4], ignore_index=True)

        for i in range(batch_end3, batch_end3+455):
            firm = ss.loc[i, 'key']
            y.append(firm_year_gram_count[firm]['CAR[0,1]'])
       
    pk = pd.concat([pk1,pk2,pk3])  
    y = pd.DataFrame(y, columns=['CAR'])

    # 머신러닝 학습과 예측 시간 측정
    start_time = time.time()
    dt_model.fit(pk, y['CAR'])
    predictions = dt_model.predict(pk)
    mse = mean_squared_error(y['CAR'], predictions) # MSE 계산
    mse_values.append(mse)
    
    # 결과 저장
    dt_results = pd.DataFrame(predictions, columns=['DT'])
    dt_results = pd.concat([y, dt_results], axis=1)
    dt_results.to_csv(str(batch_end3) + '_DT_CPU_4556_0.001.csv', encoding='cp949')
    end_time = time.time()
    total_time += (end_time - start_time)
    
    # 메모리 정리
    del pk, y, dt_results, predictions
    gc.collect()

print('총 Decision Tree_0.001 학습 시간 (CPU):', total_time)
overall_mse = np.mean(mse_values)
overall_rmse = np.sqrt(overall_mse)
print('전체 RMSE:', overall_rmse)

del dt_model
gc.collect()


총 Decision Tree_0.001 학습 시간 (CPU): 183.05551266670227
전체 RMSE: 0.07969894223663


0

In [27]:
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import time
from sklearn.metrics import mean_squared_error
import numpy as np
import gc

dt_model = DecisionTreeRegressor(ccp_alpha=0.01) #ccp_alpha=0.01 -> 프루닝시, 만일 DT 성능 종으면 프루닝 숫자 바꿔가면서 마지막으로 해보자

batch = 455*3
end = len(ss) - 455
total_time = 0
mse_values = []

for batch_start in range(0, end, batch):
    y = []
    
    batch_end1 = min(batch_start + 455, end)
    pk1 = pd.read_pickle(str(batch_end1) + "_1.pkl")
    pk1 = pk1.astype('float32')
        
    batch_end2 = min(batch_end1 + 455, end)
    pk2 = pd.read_pickle(str(batch_end2) + "_1.pkl")
    pk2 = pk2.astype('float32')
    
    batch_end3 = min(batch_end2 + 455, end)
    pk3 = pd.read_pickle(str(batch_end3) + "_1.pkl")
    pk3 = pk3.astype('float32')
   
    for i in range(batch_start, batch_end3):
        firm = ss.loc[i, 'key']
        y.append(firm_year_gram_count[firm]['CAR[0,1]'])
    
    if batch_end3 == end:
        pk4 = pd.read_pickle(str(batch_end3+455) + "_1.pkl")
        pk4 = pk4.astype('float32')
        pk3 = pd.concat([pk3, pk4], ignore_index=True)

        for i in range(batch_end3, batch_end3+455):
            firm = ss.loc[i, 'key']
            y.append(firm_year_gram_count[firm]['CAR[0,1]'])
       
    pk = pd.concat([pk1,pk2,pk3])  
    y = pd.DataFrame(y, columns=['CAR'])

    # 머신러닝 학습과 예측 시간 측정
    start_time = time.time()
    dt_model.fit(pk, y['CAR'])
    predictions = dt_model.predict(pk)
    mse = mean_squared_error(y['CAR'], predictions) # MSE 계산
    mse_values.append(mse)
    
    # 결과 저장
    dt_results = pd.DataFrame(predictions, columns=['DT'])
    dt_results = pd.concat([y, dt_results], axis=1)
    dt_results.to_csv(str(batch_end3) + '_DT_CPU_4556_0.01.csv', encoding='cp949')
    end_time = time.time()
    total_time += (end_time - start_time)
    
    # 메모리 정리
    del pk, y, dt_results, predictions
    gc.collect()

print('총 Decision Tree_0.01 학습 시간 (CPU):', total_time)
overall_mse = np.mean(mse_values)
overall_rmse = np.sqrt(overall_mse)
print('전체 RMSE:', overall_rmse)

del dt_model
gc.collect()


총 Decision Tree_0.01 학습 시간 (CPU): 183.5637948513031
전체 RMSE: 0.10640376265047138


0

In [28]:
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import time
from sklearn.metrics import mean_squared_error
import numpy as np
import gc

dt_model = DecisionTreeRegressor(ccp_alpha=0.1) #ccp_alpha=0.01 -> 프루닝시, 만일 DT 성능 종으면 프루닝 숫자 바꿔가면서 마지막으로 해보자

batch = 455*3
end = len(ss) - 455
total_time = 0
mse_values = []

for batch_start in range(0, end, batch):
    y = []
    
    batch_end1 = min(batch_start + 455, end)
    pk1 = pd.read_pickle(str(batch_end1) + "_1.pkl")
    pk1 = pk1.astype('float32')
        
    batch_end2 = min(batch_end1 + 455, end)
    pk2 = pd.read_pickle(str(batch_end2) + "_1.pkl")
    pk2 = pk2.astype('float32')
    
    batch_end3 = min(batch_end2 + 455, end)
    pk3 = pd.read_pickle(str(batch_end3) + "_1.pkl")
    pk3 = pk3.astype('float32')
   
    for i in range(batch_start, batch_end3):
        firm = ss.loc[i, 'key']
        y.append(firm_year_gram_count[firm]['CAR[0,1]'])
    
    if batch_end3 == end:
        pk4 = pd.read_pickle(str(batch_end3+455) + "_1.pkl")
        pk4 = pk4.astype('float32')
        pk3 = pd.concat([pk3, pk4], ignore_index=True)

        for i in range(batch_end3, batch_end3+455):
            firm = ss.loc[i, 'key']
            y.append(firm_year_gram_count[firm]['CAR[0,1]'])
       
    pk = pd.concat([pk1,pk2,pk3])  
    y = pd.DataFrame(y, columns=['CAR'])

    # 머신러닝 학습과 예측 시간 측정
    start_time = time.time()
    dt_model.fit(pk, y['CAR'])
    predictions = dt_model.predict(pk)
    mse = mean_squared_error(y['CAR'], predictions) # MSE 계산
    mse_values.append(mse)
    
    # 결과 저장
    dt_results = pd.DataFrame(predictions, columns=['DT'])
    dt_results = pd.concat([y, dt_results], axis=1)
    dt_results.to_csv(str(batch_end3) + '_DT_CPU_4556_0.1.csv', encoding='cp949')
    end_time = time.time()
    total_time += (end_time - start_time)
    
    # 메모리 정리
    del pk, y, dt_results, predictions
    gc.collect()

print('총 Decision Tree_0.1 학습 시간 (CPU):', total_time)
overall_mse = np.mean(mse_values)
overall_rmse = np.sqrt(overall_mse)
print('전체 RMSE:', overall_rmse)

del dt_model
gc.collect()


총 Decision Tree_0.1 학습 시간 (CPU): 181.48838877677917
전체 RMSE: 0.13663168005148477


0

In [29]:
import pandas as pd

batch = 455*3
end = len(ss) - 455
d = pd.read_csv('CAR_control.csv', encoding='cp949', index_col=0)
d1 = pd.DataFrame()
for batch_start in range(0, end, batch):
    batch_end = min(batch_start + batch, end)
    d2 = pd.read_csv(str(batch_end)+'_DT_CPU_4556.csv',index_col = 0)
    d2 = d2.rename(columns={'DT': 'DT_CPU_4553'})
    d2 = d2.drop(columns = ['CAR'])
    
    d3 = pd.read_csv(str(batch_end)+'_DT_CPU_4556_0.001.csv',index_col = 0)
    d3 = d3.rename(columns={'DT': 'DT_CPU_4553_0.001'})
    d3 = d3.drop(columns = ['CAR'])
    
    d4 = pd.read_csv(str(batch_end)+'_DT_CPU_4556_0.01.csv',index_col = 0)
    d4 = d4.rename(columns={'DT': 'DT_CPU_4553_0.01'})
    d4 = d4.drop(columns = ['CAR'])

    d5 = pd.read_csv(str(batch_end)+'_DT_CPU_4556_0.1.csv',index_col = 0)
    d5 = d5.rename(columns={'DT': 'DT_CPU_4553_0.1'})
    d5 = d5.drop(columns = ['CAR'])

    dd = pd.concat([d2, d3, d4, d5], axis=1)
    d1 = pd.concat([d1, dd], ignore_index=True)

d = pd.concat([d, d1], axis=1)
d.to_csv('CAR_control2.csv', encoding='cp949')

<center><b>머신러닝 결과 표 - 455*3 배치 추가</b></center>

<br>

|       | Time  | RMSE | 
|-------|-----------------|--------------------|
| RF(CPU)  | 5095   | 0.0453      | 
| RF3(CPU)  | 8101  | 0.0513     | 
| DT(CPU)  | 191  | 0.0011     | 
| DT0.001(CPU)  | 183  | 0.0796    | 
| DT0.01(CPU)  | 183 | 0.1064    | 
| DT0.1(CPU)  | 181  | 0.1366     | 
</table>
<br>
프루닝 조건이 Decision Tree에서 엄격해질수록 시간 단축과 RMSE가 높아지는것을 볼 수 있다. <br>
이제 이 값들을 기존 분석과 동일하게 CAR과의 회귀에 사용해보자

In [41]:
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
import pandas as pd

ddd = pd.read_csv('CAR_control2.csv', encoding='cp949', index_col=0)

X = ['RF_CPU', 'RF_CPU_4553', 'DT_CPU_4553', 'DT_CPU_4553_0.001', 'DT_CPU_4553_0.01', 'DT_CPU_4553_0.1']

models = []

for x in X:
    if '.' in x: #0.001과 같은 경우 변수명이 안읽혀서 변수명을 새로 하기보단 방법을 찾아보니 {} 형태로 감싸기로 ㅎㅁ
        formula = "CAR ~ sale + lnmkv + bperhare + Q('{}')".format(x)
    else:
        formula = 'CAR ~ sale + lnmkv + bperhare + {}'.format(x)
    model = smf.ols(formula, data=ddd).fit()
    models.append(model)
          
dfoutput = summary_col(models, stars=True, model_names=X)
print(dfoutput)



                         RF_CPU   RF_CPU_4553 DT_CPU_4553 DT_CPU_4553_0.001 DT_CPU_4553_0.01 DT_CPU_4553_0.1
------------------------------------------------------------------------------------------------------------
Intercept              -0.0004    -0.0008     -0.0000     -0.0020           0.0067***        0.0080***      
                       (0.0007)   (0.0008)    (0.0000)    (0.0018)          (0.0022)         (0.0027)       
sale                   -0.0000*   -0.0000     -0.0000     0.0000            0.0000           0.0000         
                       (0.0000)   (0.0000)    (0.0000)    (0.0000)          (0.0000)         (0.0000)       
lnmkv                  0.0001     0.0001      0.0000      0.0002            -0.0011***       -0.0012***     
                       (0.0001)   (0.0001)    (0.0000)    (0.0003)          (0.0003)         (0.0004)       
bperhare               -0.0000*** -0.0000***  0.0000      0.0000            0.0000***        0.0000***      
                  

In [42]:
ddd.head()

,CAR,GB_CPU,RF_CPU,SVR_CPU,SVR_GPU,GB_CPU_4553,RF_CPU_4553,SVR_CPU_4553,positive_ratio,negative_ratio,PN,PN2,sale,lnmkv,bperhare,DT_CPU_4553,DT_CPU_4553_0.001,DT_CPU_4553_0.01,DT_CPU_4553_0.1
0,0.062292,0.010994,0.036908,0.029830,0.030036,-0.000428,0.031726,0.027738,0.043576,0.094666,-0.369565,-0.051089,756.345,5.920677,2.8589,0.062292,-0.002546,-0.002546,-0.008655
1,-0.003595,-0.001089,0.004707,0.030952,0.031161,-0.000428,-0.006985,0.026860,0.040948,0.089799,-0.373626,-0.048851,1121.506,6.523892,3.2054,-0.003595,-0.002546,-0.002546,-0.008655
2,-0.003817,0.004312,-0.002415,0.028356,0.028570,-0.000428,-0.002817,0.023964,0.043892,0.096562,-0.375000,-0.052670,1173.024,6.379215,3.7184,-0.003817,-0.002546,-0.002546,-0.008655
3,-0.000403,0.000020,-0.004102,0.026880,0.027096,-0.000428,-0.006216,0.023439,0.045421,0.098168,-0.367347,-0.052747,1193.625,6.455379,4.3328,-0.000403,-0.002546,-0.002546,-0.008655
4,-0.073336,-0.023900,-0.039274,0.026566,0.026767,-0.000428,-0.038009,0.026761,0.043605,0.095930,-0.375000,-0.052326,1151.921,6.699709,4.8720,-0.073336,-0.002546,-0.002546,-0.008655


<oi>
DT의 유의성이 높게 나오긴 하였지만 .head()를 통해 구조를 보면 CAR 자체를 그대로 인식하는것 같다. 즉 데이터의 열값이 반영이 안되는듯하다. <br>
또한, 프루닝의 조건이 들어갈수록,  R-square가 떨어지면서 또한 모든 데이터가 같은 예측값을 가지게 된 듯 하다. <br>
즉 프루닝은 계산 복잡도를 통하여 필요한 트리만 찾아 적합 시간이 줄어든다는 장점은 있으나, 이를 바탕으로 CAR과 회귀를 할 시 정확도에 의문이 존재한다. <br>
그렇기에, 기존의 결론과 유사하게 Random Forrest의 경우 CAR과의 상관관계 분석에 유리하다는 것을 알 수 있다.